# Week6 Class Assignment Tobigs15th 이성범

In [1]:
import numpy as np

# SOLID 원칙이란? 

SOLID는 다섯 개의 원칙으로 이루어져 있다.

- SRP: Single Responsibility Principle. 단일 책임 원칙.
- OCP: Open-Closed Principle. 개방-폐쇄 원칙
- LSP: Liskov Substitution Principle. 리스코프 치환 원칙.
- ISP: Interface Segregation Principle. 인터페이스 분리 원칙.
- DIP: Dependency Inversion Principle. 의존성 역전 원칙.

SRC란 하나의 클래스는 단 한가지의 변경 이유만을 가져야 한다는 것이다.즉 하나의 클래스가 한가지의 번경 이유 만을 가져야 한다는 말은 단 하나의 책임과 기능을 가져야 한다는 것을 의미한다.

OCP란 변경이 필요할 때는 기존 코드를 수정하기 보다는 새로운 코드를 추가해야 한다는 것이다.

LSP란 하위 클래스와 상위 클래스의 위치를 변환해도 잘 동작해야 한다는 것이다.

ISP란 여러 기능이 통합되어 있는 하나의 인터페이스보다 기능별로 인터페이스를 분류하는 것이 더 좋다는 것으로 단하나의 책임과 기능을 가져야 한다는 SRC원칙과 유사하다는 것을 알 수 있다.

DIP는 상위 클래스가 하위 클래스에 종속되면 안된다는 것이다.

따라서 SOLID 원칙을 지키며 내가 만들 모델은 LinearRegression과 LogisticRegression이다. 우선 만들 클래스가 예측을 위한 기능 한가지만을 가지며 자식 클래스가 부모 클래스를 상속 받아 메소드를 오버라이딩 하여 그 기능을 구체화 시키기 때문에 SOLID 원칙을 잘 지켰다고 말할 수 있다.

우선 LinearRegression과 LogisticRegression을 만들기 전에 각 클래스에 필수적으로 들어가야 할 메소드에 대해여 생각해보자.

- 학습률, 임계치, 반복횟수, 학습내용 출력 여부 등의 파라미터를 가지는 생성자가 필요하다.
- 손실 함수를 구하는 메소드가 필요하다.
- 경사하강법을 구현하기 위해서 편미분을 통해서 기울기를 구하는 메소드가 필요하다.
- fit을 위한 메소드가 필요하다.
- predict을 위한 메소드가 필요하다.
- coef와 intercept를 출력하기 위한 메소드가 필요하다.


필요한 메소드를 살펴보면 학습률, 임계치, 반복횟수, 학습내용 출력 여부 등의 파라미터의 경우 LinearRegression과 LogisticRegression 모두 동일하게 가지는 파라미터이다.

손실 함수의 경우 LinearRegression은 MSE를 사용하고 LogisticRegression은 cross entropy를 사용하기 때문에 서로 다른 메소드를 가질 것이다.

경사하강법을 구현하기 위해 기울기를 구하는 미분 함수의 경우 LinearRegression과 LogisticRegression 모두 동일하다.

fit 함수의 경우  LinearRegression과 LogisticRegression 모두 경사하강법을 통해서 파라미터를 업데이트 하기 때문에 동일한다.

LinearRegression의 경우 연속적인 값을 에측하고, LogisticRegression의 분류를 위해 0,1의 값을 예측하기 때문에  다른 predict 메소드를 가져야 한다.

coef와 intercept의 경우 학습을 통해서 반환되는 W와 b 이기 때문에 동일한 메소드를 가진다.

위 내용을 정리하자면 동일한 메소드를 가지는 경우가 존재하기 때문에 Model이라는 부모 클래스를 미리 정의하여 코드의 중복을 제거할 필요가 있다.

따라서 우선적으로 메소드의 이름은 같지만 메소드의 구성 형식은 달라져야할 즉 메소드 오버라이딩이 되어야 할 메소드는 추상메소드로 구현할 예정이다.(ABC 패키지를 써야 하지만 임의적으로 pass를 통해서 구현했습니다.) 그리고 동일한 형식을 가져야 할 메소드의 경우 부모 클래스에 미리 구현할 예정이다.

부모 클래스인 Model을 만든 후 상속을 통해서 LinearRegression과 LogisticRegression에서 직접 구현이 필요한 메소드를 오버라이딩 하여 최종 LinearRegression과 LogisticRegression 클래스를 만들 예정이다.

## 부모 클래스

필수적으로 가져야하는 메소드를 기반으로 자식 클래스에 상속될 Model 클래스를 만들었다.

In [2]:
class Model():
    
    def __init__(self, learning_rate = 0.001, threshold = 0.01, max_iterations = 1000, verbose=False):
        self._learning_rate = learning_rate # 학습률
        self._threshold = threshold # 임계치
        self._max_iterations = max_iterations # 최대 반복 횟수
        self._verbose = verbose # 학습 내용 출력 여부
    
    # 손실 함수가 정의될 메소드
    def loss_func(self):
        pass
    
    # 기울기를 구하기 위한 미분 함수를 구현
    def numerical_derivative(self, f, x):
        delta_x = 1e-4  # 0.0001
        grad = np.zeros_like(x)
        it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])

        while not it.finished:
            idx = it.multi_index
            tmp_val = x[idx]
            x[idx] = float(tmp_val) + delta_x
            fx1 = f(x)  # f(x+delta_x)

            x[idx] = tmp_val - delta_x
            fx2 = f(x)  # f(x-delta_x)
            grad[idx] = (fx1 - fx2) / (2 * delta_x)

            x[idx] = tmp_val
            it.iternext()

        return grad
    
    # 경사하강법을 통해서 학습될 메소드
    def fit(self, X_data, y_data):
        self._W = np.random.rand(X_data.shape[1], 1)
        self._b = np.random.rand(1)

        # 미분을 위한 함수 정의
        f = lambda x : self.loss_func(X_data, y_data)

        # 경사하강법을 통한 가중치 업데이트
        for step in range(self._max_iterations):
            self._W -= self._learning_rate * self.numerical_derivative(f, self._W)
            self._b -= self._learning_rate * self.numerical_derivative(f, self._b)

            if (step % 100 == 0) and self._verbose:
                print("step = ", step, "error value = ", self.loss_func(X_data, y_data), "W = ", self._W, ", b = ", self._b)

            if self.loss_func(X_data, y_data) < self._threshold:
                print(f"지정 threshold : {self._threshold} 보다 작은 값 조기 중단 - {step}회 반복")
                break
    
    # 예측 값이 반활될 메소드
    def predict(self):
        pass
    
    # coef를 반환하는 메소드
    def get_coef(self):
        return self._W
    
    # Intercept를 반환하는 메소드
    def get_intercept(self):
        return self._b

## 자식 클래스

Model 함수에서 정의 되지 않은 loss_func과 predict 부분만을 오버라이딩하여 MyLinearRegression 클래스를 만들었다.

In [3]:
class MyLinearRegression(Model):

    def __init__(self, learning_rate=0.001, threshold=0.01, max_iterations=1000, verbose=False):
        super().__init__(learning_rate, threshold, max_iterations, verbose)

    def predict(self, x):
        result = np.dot(x, self._W) + self._b

        return result
    
    # MSE 구현
    def loss_func(self, x, y):
        y_hat = np.dot(x, self._W) + self._b
        val = np.sum((y - y_hat)**2) / 2

        return val

Model 함수에서 정의 되지 않은 loss_func과 predict 부분만을 오버라이딩하여 MyLogisticRegression을 클래스를 만들었다. 그런데 여기서 MyLinearRegression과 다른 점은 LogisticRegression의 경우 sigmoid를 기반으로 만들기 때문에 따로 sigmoid 함수를 구현했다는 점이다.

In [4]:
class MyLogisticRegression(Model):
    
    def __init__(self, learning_rate = 0.001, threshold = 0.01, max_iterations = 1000, verbose=False):
        super().__init__(learning_rate, threshold, max_iterations, verbose)

    # sigmoid구현
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # cross entropy 구현
    def loss_func(self, x, y):
        # 값이 무한으로 커지는 것을 방지
        delta = 1e-8

        z = np.dot(x, self._W) + self._b
        y_hat = self.sigmoid(z)
        val = -np.sum(y * np.log(y_hat + delta) + (1 - y) * np.log((1 - y_hat) + delta))

        return val

    def predict(self, x):
        z = np.dot(x, self._W) + self._b
        y_hat = self.sigmoid(z)
        result = np.where(y_hat <= 0.5, 0, 1)

        return result

## 구현된 모델 확인

### 학습 데이터 정의

In [5]:
X = np.array([2, 4, 6, 8, 10, 12, 14, 16, 18, 20]).reshape(-1,2)
# 회귀 예측을 위한 target data
y_R = np.array([4, 8, 12, 16, 20]).reshape(5, 1)
# 문류 예측을 위한 target data
y_L = np.array([0, 0, 0, 1, 1]).reshape(5, 1)

### 선형 모델 학습

In [6]:
lr = MyLinearRegression(threshold = 0.001)
lr.fit(X, y_R)
lr_pred = lr.predict(X)
lr_intercept = lr.get_intercept()
lr_coef = lr.get_coef()
print(f'lr_pred : {lr_pred.reshape(-1)}, lr_intercept : {lr_intercept}, lr_coef: {lr_coef.reshape(-1)}')

지정 threshold : 0.001 보다 작은 값 조기 중단 - 318회 반복
lr_pred : [ 4.03421068  8.02163412 12.00905755 15.99648099 19.98390443], lr_intercept : [0.77889457], lr_coef: [0.36605366 0.6308022 ]


값을 제대로 예측했다는 것을 알 수 있다.

### 로지스틱 모델 학습

In [7]:
lg = MyLogisticRegression(threshold = 0.001)
lg.fit(X, y_L)
lg_pred = lg.predict(X)
lg_intercept = lg.get_intercept()
lg_coef = lg.get_coef()
print(f'lg_pred : {lg_pred.reshape(-1)}, lg_intercept : {lg_intercept}, lr_coef: {lg_coef.reshape(-1)}')

lg_pred : [0 0 0 1 1], lg_intercept : [-0.3005999], lr_coef: [ 0.95989852 -0.77490331]


값을 제대로 예측했다는 것을 알 수 있다.

참고자료

- https://ko.wikipedia.org/wiki/SOLID_(%EA%B0%9D%EC%B2%B4_%EC%A7%80%ED%96%A5_%EC%84%A4%EA%B3%84)
- https://medium.com/@sunnkis/%EA%B0%9D%EC%B2%B4%EC%A7%80%ED%96%A5-%EC%84%A4%EA%B3%84-solid-%EC%9B%90%EC%B9%99-b035189988cf
- https://johngrib.github.io/wiki/SOLID/
- https://www.youtube.com/watch?v=3ELMIbGTIDs
- https://www.youtube.com/watch?v=nhzljkpjjFk